In [1]:
import pandas as pd
import json

# 处理BBJ disease

In [26]:
CT_list = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scATAC.csv')
CT_list_eQTL = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scRNA.csv')
Trait_list =pd.read_csv('/CIMA/Data/BBJ_GWAS_for_SMR_OPERA/BBJ_disease.txt',sep='\t',header=None)

In [3]:
caQTL_SMR_summary = pd.DataFrame(columns = Trait_list[0].to_list(),index = CT_list['final_annotation'].to_list())
caQTL_SMR_thershold = pd.DataFrame(columns = Trait_list[0].to_list(),index = CT_list['final_annotation'].to_list())
caQTL_SMR_thershold = caQTL_SMR_thershold.astype(float)
sig_caPeak_df = pd.DataFrame()
sig_caPeak_list = {}
sig_caPeak_full = pd.DataFrame()

for trait in Trait_list[0].to_list():
    for celltype in CT_list['final_annotation'].to_list():
        SMR_result = pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/BBJ/{trait}/caQTLsmr_{celltype}.smr',sep = '\t')
        SMR_result = SMR_result.dropna(subset=['p_SMR'])
        if SMR_result.shape[0] > 0: 
            caQTL_SMR_thershold.loc[celltype,trait] = 0.05/SMR_result.shape[0]
            SMR_result = SMR_result[SMR_result['p_SMR'] < 0.05/SMR_result.shape[0]]
            SMR_result = SMR_result.dropna(subset=['p_HEIDI'])
            SMR_result = SMR_result[SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            SMR_result = SMR_result[SMR_result['p_GWAS'] < 1e-5]
            sig_count = len(SMR_result['probeID'].unique())
            caQTL_SMR_summary.loc[celltype,trait] = sig_count
            sig_caPeak_list[f'{celltype}***{trait}'] = list(SMR_result['probeID'].unique())
            if SMR_result.shape[0] > 0:
                SMR_result_sig = SMR_result.copy()
                SMR_result_sig.loc[:,'celltype'] = celltype
                SMR_result_sig.loc[:,'trait'] = trait
                sig_caPeak_full = pd.concat([sig_caPeak_full,SMR_result_sig],ignore_index = True)
                SMR_result_sig = SMR_result_sig.loc[:,['probeID','topSNP','celltype','trait']]
                sig_caPeak_df = pd.concat([sig_caPeak_df,SMR_result_sig],ignore_index = True)       
        else:
            caQTL_SMR_summary.loc[celltype,trait] = 0
            caQTL_SMR_thershold.loc[celltype,trait] = np.nan
            sig_caPeak_list[f'{celltype}***{trait}'] = []

sig_caPeak_full.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_BBJ_diease_SMR_caPeak_full.csv')
caQTL_SMR_summary.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_BBJ_diease_SMR_summary.csv')
caQTL_SMR_thershold.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_BBJ_diease_SMR_thershold.csv')
sig_caPeak_df.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_BBJ_diease_SMR_caPeak.csv')
with open('/CIMA/Result/downstream/SMR_summary/sig_caPeak_smr_BBJ_disease.json', 'w') as f:
    json.dump(sig_caPeak_list, f)

In [4]:
eQTL_SMR_summary = pd.DataFrame(columns = Trait_list[0].to_list(),index = CT_list_eQTL['final_annotation'].to_list())
eQTL_SMR_thershold = pd.DataFrame(columns = Trait_list[0].to_list(),index = CT_list_eQTL['final_annotation'].to_list())
eQTL_SMR_thershold = eQTL_SMR_thershold.astype(float)
sig_eGene_list = {}
sig_eGene_df = pd.DataFrame()
sig_eGene_full = pd.DataFrame()

for trait in Trait_list[0].to_list():
    for celltype in CT_list_eQTL['final_annotation'].to_list():
        SMR_result = pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/BBJ/{trait}/eQTLsmr_{celltype}.smr',sep = '\t')
        SMR_result = SMR_result.dropna(subset=['p_SMR'])
        if SMR_result.shape[0] > 0:
            eQTL_SMR_thershold.loc[celltype,trait] = 0.05/SMR_result.shape[0]
            SMR_result = SMR_result[SMR_result['p_SMR'] < 0.05/SMR_result.shape[0]]
            SMR_result = SMR_result.dropna(subset=['p_HEIDI'])
            SMR_result = SMR_result[SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            SMR_result = SMR_result[SMR_result['p_GWAS'] < 1e-5]
            sig_count = len(SMR_result['probeID'].unique())
            eQTL_SMR_summary.loc[celltype,trait] = sig_count
            sig_eGene_list[f'{celltype}***{trait}'] = list(SMR_result['probeID'].unique())
            if SMR_result.shape[0] > 0:
                SMR_result_sig = SMR_result.copy()
                SMR_result_sig.loc[:,'celltype'] = celltype
                SMR_result_sig.loc[:,'trait'] = trait
                sig_eGene_full = pd.concat([sig_eGene_full,SMR_result_sig],ignore_index = True)
                SMR_result_sig = SMR_result_sig.loc[:,['probeID','topSNP','celltype','trait']]
                sig_eGene_df = pd.concat([sig_eGene_df,SMR_result_sig],ignore_index = True)
           
        else:
            eQTL_SMR_summary.loc[celltype,trait] = 0
            eQTL_SMR_thershold.loc[celltype,trait] = np.nan
            sig_eGene_list[f'{celltype}***{trait}'] = []

sig_eGene_full.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_BBJ_diease_SMR_eGene_full.csv')
eQTL_SMR_summary.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_BBJ_diease_SMR_summary.csv')
eQTL_SMR_thershold.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_BBJ_diease_SMR_thershold.csv')
sig_eGene_df.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_BBJ_diease_SMR_eGene.csv')
with open('/CIMA/Result/downstream/SMR_summary/sig_eGene_smr_BBJ_disease.json', 'w') as f:
    json.dump(sig_eGene_list, f)

In [5]:
xQTL_SMR_summary = pd.DataFrame(columns = Trait_list[0].to_list(),index = CT_list['final_annotation'].to_list())
xQTL_SMR_summary.fillna(0, inplace=True)

merge_result = pd.DataFrame()
for trait in Trait_list[0].to_list():
    for celltype in CT_list['final_annotation'].to_list():
        caQTL_SMR_result = pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/BBJ/{trait}/caQTLsmr_{celltype}.smr',sep = '\t')
        caQTL_SMR_result = caQTL_SMR_result.dropna(subset=['p_SMR'])
        eQTL_SMR_result =  pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/BBJ/{trait}/eQTLsmr_{celltype}.smr',sep = '\t')
        eQTL_SMR_result = eQTL_SMR_result.dropna(subset=['p_SMR'])

        if caQTL_SMR_result.shape[0] > 0: 
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_SMR'] < 0.05/caQTL_SMR_result.shape[0]]
            #caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_SMR'] < 0.05]
            caQTL_SMR_result = caQTL_SMR_result.dropna(subset=['p_HEIDI'])
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_GWAS'] < 1e-5]
        
        if eQTL_SMR_result.shape[0] > 0: 
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_SMR'] < 0.05/eQTL_SMR_result.shape[0]]
            #eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_SMR'] < 0.05]
            eQTL_SMR_result = eQTL_SMR_result.dropna(subset=['p_HEIDI'])
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_GWAS'] < 1e-5]
        
        # 如果两个结果都不为空，进行交集操作
        if caQTL_SMR_result.shape[0] > 0 and eQTL_SMR_result.shape[0] > 0:
            # 找共同的SNPs
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['topSNP'].isin(eQTL_SMR_result['topSNP'])]
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['topSNP'].isin(caQTL_SMR_result['topSNP'])]
            
            # 合并结果
            if not caQTL_SMR_result.empty and not eQTL_SMR_result.empty:
                xQTL_SMR_summary.loc[celltype,trait] = 1
                combined_result = pd.concat([caQTL_SMR_result, eQTL_SMR_result], ignore_index=True)
                combined_result.loc[:,'celltype'] = celltype
                combined_result.loc[:,'trait'] = trait
                merge_result = pd.concat([merge_result,combined_result],ignore_index = True)

merge_result.to_csv('/CIMA/Result/downstream/SMR_summary/xQTL_BBJ_diease_SMR_share.csv')
xQTL_SMR_summary.to_csv('/CIMA/Result/downstream/SMR_summary/xQTL_BBJ_summary.csv')

/tmp/ipykernel_3822141/1852507540.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  xQTL_SMR_summary.fillna(0, inplace=True)


# 处理lime

In [12]:
CT_list = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scATAC.csv')
CT_list_eQTL = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scRNA.csv')

Trait_lipid =pd.read_csv('/CIMA/Result/GWAS/sig_lipid.csv')
Trait_meta =pd.read_csv('/CIMA/Result/GWAS/sig_meta.csv')
Trait_list = pd.concat([Trait_lipid,Trait_meta],axis=0)
Trait_list = Trait_list[['V2','V1']]

In [13]:
Trait_dict = Trait_list.set_index('V2')['V1'].to_dict()

In [21]:
Trait_dict

{'lipid_490': 'LPE 22:6',
 'lipid_505': 'LPC 20:4',
 'lipid_524': 'PE 18:2-20:1',
 'lipid_585': 'PC 18:2-18:2',
 'META_35': '2-HYDROXY-4-(METHYLTHIO)BUTANOATE',
 'META_36': 'Phenylglyoxylic acid',
 'META_37': 'D-Arabinose',
 'META_86': '3-Indoleglyoxylic acid',
 'META_270': 'Beta-Alanine',
 'META_288': '7-Methylguanosine',
 'META_292': '3-Indolepropionic acid'}

In [14]:
caQTL_SMR_summary = pd.DataFrame(columns = Trait_list['V2'].to_list(),index = CT_list['final_annotation'].to_list())
caQTL_SMR_thershold = pd.DataFrame(columns = Trait_list['V2'].to_list(),index = CT_list['final_annotation'].to_list())
caQTL_SMR_thershold = caQTL_SMR_thershold.astype(float)
sig_caPeak_df = pd.DataFrame()
sig_caPeak_list = {}
sig_caPeak_full = pd.DataFrame()

for trait in Trait_list['V2'].to_list():
    for celltype in CT_list['final_annotation'].to_list():
        SMR_result = pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/lipid_meta/{trait}/caQTLsmr_{celltype}.smr',sep = '\t')
        SMR_result = SMR_result.dropna(subset=['p_SMR'])
        if SMR_result.shape[0] > 0: 
            caQTL_SMR_thershold.loc[celltype,trait] = 0.05/SMR_result.shape[0]
            SMR_result = SMR_result[SMR_result['p_SMR'] < 0.05/SMR_result.shape[0]]
            SMR_result = SMR_result.dropna(subset=['p_HEIDI'])
            SMR_result = SMR_result[SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            SMR_result = SMR_result[SMR_result['p_GWAS'] < 1e-5]
            sig_count = len(SMR_result['probeID'].unique())
            caQTL_SMR_summary.loc[celltype,trait] = sig_count
            sig_caPeak_list[f'{celltype}***{trait}'] = list(SMR_result['probeID'].unique())
            if SMR_result.shape[0] > 0:
                SMR_result_sig = SMR_result.copy()
                SMR_result_sig.loc[:,'celltype'] = celltype
                SMR_result_sig.loc[:,'trait'] = trait
                sig_caPeak_full = pd.concat([sig_caPeak_full,SMR_result_sig],ignore_index = True)
                SMR_result_sig = SMR_result_sig.loc[:,['probeID','topSNP','celltype','trait']]
                sig_caPeak_df = pd.concat([sig_caPeak_df,SMR_result_sig],ignore_index = True)
            
        else:
            caQTL_SMR_summary.loc[celltype,trait] = 0
            caQTL_SMR_thershold.loc[celltype,trait] = np.nan
            sig_caPeak_list[f'{celltype}***{trait}'] = []

In [15]:
sig_caPeak_full['trait'] = sig_caPeak_full['trait'].map(Trait_dict)
caQTL_SMR_summary.columns = caQTL_SMR_summary.columns.map(Trait_dict)
caQTL_SMR_thershold.columns = caQTL_SMR_thershold.columns.map(Trait_dict)
sig_caPeak_df['trait'] = sig_caPeak_df['trait'].map(Trait_dict)

sig_caPeak_full.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_lime_sig_SMR_caPeak_full.csv')
caQTL_SMR_summary.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_lime_SMR_summary.csv')
caQTL_SMR_thershold.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_lime_SMR_thershold.csv')
sig_caPeak_df.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_lime_sig_SMR_caPeak.csv')
with open('/CIMA/Result/downstream/SMR_summary/sig_caPeak_smr_lime.json', 'w') as f:
    json.dump(sig_caPeak_list, f)

In [16]:
eQTL_SMR_summary =  pd.DataFrame(columns = Trait_list['V2'].to_list(),index = CT_list_eQTL['final_annotation'].to_list())
eQTL_SMR_thershold =  pd.DataFrame(columns = Trait_list['V2'].to_list(),index = CT_list_eQTL['final_annotation'].to_list())
eQTL_SMR_thershold = eQTL_SMR_thershold.astype(float)
sig_eGene_list = {}
sig_eGene_df = pd.DataFrame()
sig_eGene_full = pd.DataFrame()

In [17]:
for trait in Trait_list['V2'].to_list():
    for celltype in CT_list_eQTL['final_annotation'].to_list():
        SMR_result = pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/lipid_meta/{trait}/eQTLsmr_{celltype}.smr',sep = '\t')
        if SMR_result.shape[0] > 0:
            eQTL_SMR_thershold.loc[celltype,trait] = 0.05/SMR_result.shape[0]
            SMR_result = SMR_result[SMR_result['p_SMR'] < 0.05/SMR_result.shape[0]]

            SMR_result = SMR_result.dropna(subset=['p_HEIDI'])
            SMR_result = SMR_result[SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            SMR_result = SMR_result[SMR_result['p_GWAS'] < 1e-5]
            sig_count = len(SMR_result['probeID'].unique())
            eQTL_SMR_summary.loc[celltype,trait] = sig_count
            sig_eGene_list[f'{celltype}***{trait}'] = list(SMR_result['probeID'].unique())
            if SMR_result.shape[0] > 0:
                SMR_result_sig = SMR_result.copy()
                SMR_result_sig.loc[:,'celltype'] = celltype
                SMR_result_sig.loc[:,'trait'] = trait
                sig_eGene_full = pd.concat([sig_eGene_full,SMR_result_sig],ignore_index = True)
                SMR_result_sig = SMR_result_sig.loc[:,['probeID','topSNP','celltype','trait']]
                sig_eGene_df = pd.concat([sig_eGene_df,SMR_result_sig],ignore_index = True)
           
        else:
            eQTL_SMR_summary.loc[celltype,trait] = 0
            eQTL_SMR_thershold.loc[celltype,trait] = np.nan
            sig_eGene_list[f'{celltype}***{trait}'] = []

In [18]:
sig_eGene_full['trait'] = sig_eGene_full['trait'].map(Trait_dict)
eQTL_SMR_summary.columns = eQTL_SMR_summary.columns.map(Trait_dict)
eQTL_SMR_thershold.columns = eQTL_SMR_thershold.columns.map(Trait_dict)
sig_eGene_df['trait'] = sig_eGene_df['trait'].map(Trait_dict)

sig_eGene_full.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_lime_sig_SMR_eGene_full.csv')
eQTL_SMR_summary.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_lime_SMR_summary.csv')
eQTL_SMR_thershold.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_lime_SMR_thershold.csv')
sig_eGene_df.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_lime_sig_SMR_eGene.csv')
with open('/CIMA/Result/downstream/SMR_summary/sig_eGene_smr_lime.json', 'w') as f:
    json.dump(sig_eGene_list, f)

In [20]:
xQTL_SMR_summary = pd.DataFrame(columns = Trait_list['V2'].to_list(),index = CT_list['final_annotation'].to_list())
xQTL_SMR_summary.fillna(0, inplace=True)

merge_result = pd.DataFrame()
for trait in Trait_list['V2'].to_list():
    for celltype in CT_list['final_annotation'].to_list():
        caQTL_SMR_result = pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/lipid_meta/{trait}/caQTLsmr_{celltype}.smr',sep = '\t')
        caQTL_SMR_result = caQTL_SMR_result.dropna(subset=['p_SMR'])
        eQTL_SMR_result = pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/lipid_meta/{trait}/eQTLsmr_{celltype}.smr',sep = '\t')
        eQTL_SMR_result = eQTL_SMR_result.dropna(subset=['p_SMR'])

        if caQTL_SMR_result.shape[0] > 0: 
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_SMR'] < 0.05/caQTL_SMR_result.shape[0]]
            #caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_SMR'] < 0.05]
            caQTL_SMR_result = caQTL_SMR_result.dropna(subset=['p_HEIDI'])
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_GWAS'] < 1e-5]
            
        if eQTL_SMR_result.shape[0] > 0: 
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_SMR'] < 0.05/eQTL_SMR_result.shape[0]]
            #eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_SMR'] < 0.05]
            eQTL_SMR_result = eQTL_SMR_result.dropna(subset=['p_HEIDI'])
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_GWAS'] < 1e-5]
        
        # 如果两个结果都不为空，进行交集操作
        if caQTL_SMR_result.shape[0] > 0 and eQTL_SMR_result.shape[0] > 0:
            # 找共同的SNPs
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['topSNP'].isin(eQTL_SMR_result['topSNP'])]
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['topSNP'].isin(caQTL_SMR_result['topSNP'])]
            
            # 合并结果
            if not caQTL_SMR_result.empty and not eQTL_SMR_result.empty:
                xQTL_SMR_summary.loc[celltype,trait] = 1
                combined_result = pd.concat([caQTL_SMR_result, eQTL_SMR_result], ignore_index=True)
                combined_result.loc[:,'celltype'] = celltype
                combined_result.loc[:,'trait'] = trait
                merge_result = pd.concat([merge_result,combined_result],ignore_index = True)

merge_result['trait'] = merge_result['trait'].map(Trait_dict)
merge_result.to_csv('/CIMA/Result/downstream/SMR_summary/xQTL_lime_SMR_share.csv')
xQTL_SMR_summary.columns = xQTL_SMR_summary.columns.map(Trait_dict)
xQTL_SMR_summary.to_csv('/CIMA/Result/downstream/SMR_summary/xQTL_lime_SMR_summary.csv')

/tmp/ipykernel_4044350/767489059.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  xQTL_SMR_summary.fillna(0, inplace=True)


# 处理xQTL之间的SMR

In [2]:
CT_list = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scATAC.csv')
xQTL_SMR_summary = pd.DataFrame(columns = ['sig_eGene_count','sig_caPeak_count','sig_pair_counts'],index = CT_list["final_annotation"].to_list())

#只筛选topSNP为lead_caQTL的结果
sig_xQTL_SMR_full = pd.DataFrame()
for celltype in CT_list['final_annotation'].to_list():
    caPeak = pd.read_csv('/CIMA/Result/20250108_cis_caQTL_studywise_sig.csv',index_col=0)
    caPeak = caPeak[caPeak['celltype'] == celltype]
    SMR_result = pd.read_csv(f'/CIMA/Result/xQTL_SMR/{celltype}.smr',sep='\t')
    # lead caQTL
    #print(SMR_result.shape[0])
    SMR_result = SMR_result[SMR_result['topSNP'].isin(caPeak['variant_id'])]
    #print(SMR_result.shape[0])
    SMR_result = SMR_result.dropna(subset=['p_SMR'])
    #SMR_result = SMR_result[abs(SMR_result['Expo_bp'] - SMR_result['Outco_bp']) <= 50000]
    #多重检验矫正
    if SMR_result.shape[0] > 0: 
        SMR_result = SMR_result[SMR_result['p_SMR'] < 0.05/SMR_result.shape[0]]
        #SMR_result['adjusted_p_SMR'] = multipletests(SMR_result['p_SMR'], method='fdr_bh')[1]
        #SMR_result = SMR_result[SMR_result['adjusted_p_SMR'] < 0.05]
        SMR_result = SMR_result.dropna(subset=['p_HEIDI'])
        SMR_result = SMR_result[SMR_result['p_HEIDI'] > 0.01]
        #SMR_result.to_csv(f'/hwfssz5/ST_HEALTH/P20Z10200N0041/huangzhuoli/genetics_Result/Downstream_result/caQTL_eQTL_SMR_sig/{celltype}.smr',sep='\t',index = None)
        
        sig_eGene_count = len(SMR_result['Outco_ID'].unique())
        sig_caPeak_count = len(SMR_result['Expo_ID'].unique())
        sig_pair_counts = SMR_result.shape[0]

        xQTL_SMR_summary.loc[celltype,'sig_eGene_count'] = sig_eGene_count
        xQTL_SMR_summary.loc[celltype,'sig_caPeak_count'] = sig_caPeak_count
        xQTL_SMR_summary.loc[celltype,'sig_pair_counts'] = sig_pair_counts
        SMR_result['celltype'] = celltype
        sig_xQTL_SMR_full = pd.concat([sig_xQTL_SMR_full,SMR_result],ignore_index = True)
    else : 
        xQTL_SMR_summary.loc[celltype,'sig_eGene_count'] = 0
        xQTL_SMR_summary.loc[celltype,'sig_caPeak_count'] = 0
        xQTL_SMR_summary.loc[celltype,'sig_pair_counts'] = 0
        SMR_result['celltype'] = celltype
        sig_xQTL_SMR_full = pd.concat([sig_xQTL_SMR_full,SMR_result],ignore_index = True)

sig_xQTL_SMR_full['pair'] = sig_xQTL_SMR_full['Expo_ID']+'_'+sig_xQTL_SMR_full['Outco_ID']

sig_xQTL_SMR_full.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_eQTL_SMR.csv')
xQTL_SMR_summary.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_eQTL_SMR_summary.csv')

# 处理dynamic_QTL

In [7]:
sig_eGene_full = pd.DataFrame()
Trait_list =pd.read_csv('/CIMA/Data/BBJ_GWAS_for_SMR_OPERA/BBJ_disease.txt',sep='\t',header=None)
for trait in Trait_list[0].to_list():
    SMR_result = pd.read_csv(f'/CIMA/Result/dynamic/SMR_BBJ/{trait}/eQTLsmr_B.smr',sep = '\t')
    SMR_result = SMR_result.dropna(subset=['p_SMR'])
    if SMR_result.shape[0] > 0:
        #eQTL_SMR_thershold.loc[celltype,trait] = 0.05/SMR_result.shape[0]
        SMR_result = SMR_result[SMR_result['p_SMR'] < 0.05/SMR_result.shape[0]]

        SMR_result = SMR_result.dropna(subset=['p_HEIDI'])
        SMR_result = SMR_result[SMR_result['p_HEIDI'] > 0.01]
        #添加对p_GWAS的筛选
        SMR_result = SMR_result[SMR_result['p_GWAS'] < 1e-5]
        
        sig_count = len(SMR_result['probeID'].unique())
        #eQTL_SMR_summary.loc[celltype,trait] = sig_count
        #sig_eGene_list[f'{celltype}***{trait}'] = list(SMR_result['probeID'].unique())
        if SMR_result.shape[0] > 0:
            SMR_result_sig = SMR_result.copy()
            SMR_result_sig.loc[:,'trait'] = trait
            sig_eGene_full = pd.concat([sig_eGene_full,SMR_result_sig],ignore_index = True)

In [8]:
sig_eGene_full.to_csv('/CIMA/Result/downstream/SMR_summary/dynamic_eQTL_B.csv')

In [9]:
sig_eGene_full = pd.DataFrame()
Trait_list =pd.read_csv('/CIMA/Data/BBJ_GWAS_for_SMR_OPERA/BBJ_disease.txt',sep='\t',header=None)
for trait in Trait_list[0].to_list():
    SMR_result = pd.read_csv(f'/CIMA/Result/dynamic/SMR_BBJ/{trait}/eQTLsmr_Monocyte.smr',sep = '\t')
    SMR_result = SMR_result.dropna(subset=['p_SMR'])
    if SMR_result.shape[0] > 0:
        #eQTL_SMR_thershold.loc[celltype,trait] = 0.05/SMR_result.shape[0]
        SMR_result = SMR_result[SMR_result['p_SMR'] < 0.05/SMR_result.shape[0]]

        SMR_result = SMR_result.dropna(subset=['p_HEIDI'])
        SMR_result = SMR_result[SMR_result['p_HEIDI'] > 0.01]
        #添加对p_GWAS的筛选
        SMR_result = SMR_result[SMR_result['p_GWAS'] < 1e-5]
        
        sig_count = len(SMR_result['probeID'].unique())
        #eQTL_SMR_summary.loc[celltype,trait] = sig_count
        #sig_eGene_list[f'{celltype}***{trait}'] = list(SMR_result['probeID'].unique())
        if SMR_result.shape[0] > 0:
            SMR_result_sig = SMR_result.copy()
            SMR_result_sig.loc[:,'trait'] = trait
            sig_eGene_full = pd.concat([sig_eGene_full,SMR_result_sig],ignore_index = True)

In [10]:
sig_eGene_full.to_csv('/CIMA/Result/downstream/SMR_summary/dynamic_eQTL_Monocyte.csv')

# 处理pQTL

In [2]:
CT_list = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scATAC.csv')
CT_list_eQTL = pd.read_csv('/CIMA/Data/20241230_xQTL_cell_sample_number/20241230_celltype_70_scRNA.csv')
Trait_list = pd.read_csv('/CIMA/Data/pQTL_ma/pQTL_list.txt',sep='\t',header=None)

In [3]:
#p_GWAS阈值1e-5
caQTL_SMR_summary = pd.DataFrame(columns = Trait_list[0].to_list(),index = CT_list['final_annotation'].to_list())
caQTL_SMR_thershold = pd.DataFrame(columns = Trait_list[0].to_list(),index = CT_list['final_annotation'].to_list())
caQTL_SMR_thershold = caQTL_SMR_thershold.astype(float)

sig_caPeak_df = pd.DataFrame()
sig_caPeak_list = {}
sig_caPeak_full = pd.DataFrame()

for trait in Trait_list[0].to_list():
    for celltype in CT_list['final_annotation'].to_list():
        SMR_result = pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/pQTL/{trait}/caQTLsmr_{celltype}.smr',sep = '\t')
        SMR_result = SMR_result.dropna(subset=['p_SMR'])
        if SMR_result.shape[0] > 0: 
            caQTL_SMR_thershold.loc[celltype,trait] = 0.05/SMR_result.shape[0]
            SMR_result = SMR_result[SMR_result['p_SMR'] < 0.05/SMR_result.shape[0]]
            SMR_result = SMR_result.dropna(subset=['p_HEIDI'])
            SMR_result = SMR_result[SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            SMR_result = SMR_result[SMR_result['p_GWAS'] < 1e-5]
            
            sig_count = len(SMR_result['probeID'].unique())
            caQTL_SMR_summary.loc[celltype,trait] = sig_count
            sig_caPeak_list[f'{celltype}***{trait}'] = list(SMR_result['probeID'].unique())
            if SMR_result.shape[0] > 0:
                SMR_result_sig = SMR_result.copy()
                SMR_result_sig.loc[:,'celltype'] = celltype
                SMR_result_sig.loc[:,'trait'] = trait
                sig_caPeak_full = pd.concat([sig_caPeak_full,SMR_result_sig],ignore_index = True)
                SMR_result_sig = SMR_result_sig.loc[:,['probeID','topSNP','celltype','trait']]
                sig_caPeak_df = pd.concat([sig_caPeak_df,SMR_result_sig],ignore_index = True)       
        else:
            caQTL_SMR_summary.loc[celltype,trait] = 0
            caQTL_SMR_thershold.loc[celltype,trait] = np.nan
            sig_caPeak_list[f'{celltype}***{trait}'] = []

sig_caPeak_full.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_pQTL_SMR_caPeak_full.csv')
caQTL_SMR_summary.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_pQTL_SMR_summary.csv')
caQTL_SMR_thershold.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_pQTL_SMR_thershold.csv')
sig_caPeak_df.to_csv('/CIMA/Result/downstream/SMR_summary/caQTL_pQTL_SMR_caPeak.csv')
with open('/CIMA/Result/downstream/SMR_summary/sig_caPeak_smr_pQTL.json', 'w') as f:
    json.dump(sig_caPeak_list, f)

In [4]:
eQTL_SMR_summary = pd.DataFrame(columns = Trait_list[0].to_list(),index = CT_list_eQTL['final_annotation'].to_list())
eQTL_SMR_thershold = pd.DataFrame(columns = Trait_list[0].to_list(),index = CT_list_eQTL['final_annotation'].to_list())
eQTL_SMR_thershold = eQTL_SMR_thershold.astype(float)
sig_eGene_list = {}
sig_eGene_df = pd.DataFrame()
sig_eGene_full = pd.DataFrame()

for trait in Trait_list[0].to_list():
    for celltype in CT_list_eQTL['final_annotation'].to_list():
        SMR_result = pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/pQTL/{trait}/eQTLsmr_{celltype}.smr',sep = '\t')
        SMR_result = SMR_result.dropna(subset=['p_SMR'])
        if SMR_result.shape[0] > 0:
            eQTL_SMR_thershold.loc[celltype,trait] = 0.05/SMR_result.shape[0]
            SMR_result = SMR_result[SMR_result['p_SMR'] < 0.05/SMR_result.shape[0]]

            SMR_result = SMR_result.dropna(subset=['p_HEIDI'])
            SMR_result = SMR_result[SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            SMR_result = SMR_result[SMR_result['p_GWAS'] < 1e-5]
            
            sig_count = len(SMR_result['probeID'].unique())
            eQTL_SMR_summary.loc[celltype,trait] = sig_count
            sig_eGene_list[f'{celltype}***{trait}'] = list(SMR_result['probeID'].unique())
            if SMR_result.shape[0] > 0:
                SMR_result_sig = SMR_result.copy()
                SMR_result_sig.loc[:,'celltype'] = celltype
                SMR_result_sig.loc[:,'trait'] = trait
                sig_eGene_full = pd.concat([sig_eGene_full,SMR_result_sig],ignore_index = True)
                SMR_result_sig = SMR_result_sig.loc[:,['probeID','topSNP','celltype','trait']]
                sig_eGene_df = pd.concat([sig_eGene_df,SMR_result_sig],ignore_index = True)
           
        else:
            eQTL_SMR_summary.loc[celltype,trait] = 0
            eQTL_SMR_thershold.loc[celltype,trait] = np.nan
            sig_eGene_list[f'{celltype}***{trait}'] = []

sig_eGene_full.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_pQTL_SMR_eGene_full.csv')
eQTL_SMR_summary.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_pQTL_SMR_summary.csv')
eQTL_SMR_thershold.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_pQTL_SMR_thershold.csv')
sig_eGene_df.to_csv('/CIMA/Result/downstream/SMR_summary/eQTL_pQTL_SMR_eGene.csv')
with open('/CIMA/Result/downstream/SMR_summary/sig_eGene_smr_pQTL.json', 'w') as f:
    json.dump(sig_eGene_list, f)

In [5]:
xQTL_SMR_summary = pd.DataFrame(columns = Trait_list[0].to_list(),index = CT_list['final_annotation'].to_list())
xQTL_SMR_summary.fillna(0, inplace=True)

merge_result = pd.DataFrame()
for trait in Trait_list[0].to_list():
    for celltype in CT_list['final_annotation'].to_list():
        caQTL_SMR_result = pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/pQTL/{trait}/caQTLsmr_{celltype}.smr',sep = '\t')
        caQTL_SMR_result = caQTL_SMR_result.dropna(subset=['p_SMR'])
        eQTL_SMR_result =  pd.read_csv(f'/CIMA/Result/xQTL_GWAS_SMR/pQTL/{trait}/eQTLsmr_{celltype}.smr',sep = '\t')
        eQTL_SMR_result = eQTL_SMR_result.dropna(subset=['p_SMR'])

        if caQTL_SMR_result.shape[0] > 0: 
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_SMR'] < 0.05/caQTL_SMR_result.shape[0]]
            #caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_SMR'] < 0.05]
            caQTL_SMR_result = caQTL_SMR_result.dropna(subset=['p_HEIDI'])
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['p_GWAS'] < 1e-5]
        
        if eQTL_SMR_result.shape[0] > 0: 
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_SMR'] < 0.05/eQTL_SMR_result.shape[0]]
            #eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_SMR'] < 0.05]
            eQTL_SMR_result = eQTL_SMR_result.dropna(subset=['p_HEIDI'])
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_HEIDI'] > 0.01]
            #添加对p_GWAS的筛选
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['p_GWAS'] < 1e-5]
            
        # 如果两个结果都不为空，进行交集操作
        if caQTL_SMR_result.shape[0] > 0 and eQTL_SMR_result.shape[0] > 0:
            # 找共同的SNPs
            caQTL_SMR_result = caQTL_SMR_result[caQTL_SMR_result['topSNP'].isin(eQTL_SMR_result['topSNP'])]
            eQTL_SMR_result = eQTL_SMR_result[eQTL_SMR_result['topSNP'].isin(caQTL_SMR_result['topSNP'])]
            
            # 合并结果
            if not caQTL_SMR_result.empty and not eQTL_SMR_result.empty:
                xQTL_SMR_summary.loc[celltype,trait] = 1
                combined_result = pd.concat([caQTL_SMR_result, eQTL_SMR_result], ignore_index=True)
                combined_result.loc[:,'celltype'] = celltype
                combined_result.loc[:,'trait'] = trait
                merge_result = pd.concat([merge_result,combined_result],ignore_index = True)

merge_result.to_csv('/CIMA/Result/downstream/SMR_summary/xQTL_pQTL_SMR_share.csv')
xQTL_SMR_summary.to_csv('/CIMA/Result/downstream/SMR_summary/xQTL_pQTL_SMR_summary.csv')

/tmp/ipykernel_3825891/3922234343.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  xQTL_SMR_summary.fillna(0, inplace=True)


In [9]:
sig_eGene_full['p_GWAS'].sort_values(ascending=False)

23     8.750000e-06
25     8.750000e-06
24     8.750000e-06
29     4.510000e-06
37     4.510000e-06
           ...     
50     1.520000e-23
49     1.520000e-23
5      1.910000e-24
275    2.150000e-36
290    4.630000e-47
Name: p_GWAS, Length: 341, dtype: float64

In [20]:
'''
import os

# 获取当前目录
current_dir = '/CIMA/Result/xQTL_GWAS_SMR/pQTL/'

# 遍历当前目录下的所有子文件夹
for subfolder in os.listdir(current_dir):
    subfolder_path = os.path.join(current_dir, subfolder)
    if os.path.isdir(subfolder_path):
        # 获取子文件夹下的文件数量
        file_count = len([f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))])
        print(f"文件夹 '{subfolder}' 中的文件数量: {file_count}")

import os

# 创建一个空字典来存储结果
unique_files_dict = {}

# 遍历当前目录下的所有子文件夹
for dirpath, dirnames, filenames in os.walk('/CIMA/Result/xQTL_GWAS_SMR/pQTL/'):
    # 只处理文件夹下的 .snp_failed_freq_ck.list 文件
    for filename in filenames:
        if filename.endswith('.snp_failed_freq_ck.list'):
            # 去掉 'caQTLsmr_' 或 'eQTLsmr_' 字段
            new_filename = filename.replace('caQTLsmr_', '').replace('eQTLsmr_', '').replace('.snp_failed_freq_ck.list','')
            
            # 存储唯一文件名，字典中的值是文件名列表
            if new_filename not in unique_files_dict:
                unique_files_dict[new_filename] = []
            
            # 存储该文件所在的子文件夹路径
            unique_files_dict[new_filename].append(dirpath)

# 为字典中的每个键计算对应值（路径列表）的长度
length_dict = {filename: len(paths) for filename, paths in unique_files_dict.items()}

# 输出新的字典
print("字典中每个文件名的路径数量:")
for filename, length in length_dict.items():
    print(f"文件名: {filename}, 路径数量: {length}")
'''

'\nimport os\n\n# 获取当前目录\ncurrent_dir = \'/CIMA/Result/xQTL_GWAS_SMR/pQTL/\'\n\n# 遍历当前目录下的所有子文件夹\nfor subfolder in os.listdir(current_dir):\n    subfolder_path = os.path.join(current_dir, subfolder)\n    if os.path.isdir(subfolder_path):\n        # 获取子文件夹下的文件数量\n        file_count = len([f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))])\n        print(f"文件夹 \'{subfolder}\' 中的文件数量: {file_count}")\n\nimport os\n\n# 创建一个空字典来存储结果\nunique_files_dict = {}\n\n# 遍历当前目录下的所有子文件夹\nfor dirpath, dirnames, filenames in os.walk(\'/CIMA/Result/xQTL_GWAS_SMR/pQTL/\'):\n    # 只处理文件夹下的 .snp_failed_freq_ck.list 文件\n    for filename in filenames:\n        if filename.endswith(\'.snp_failed_freq_ck.list\'):\n            # 去掉 \'caQTLsmr_\' 或 \'eQTLsmr_\' 字段\n            new_filename = filename.replace(\'caQTLsmr_\', \'\').replace(\'eQTLsmr_\', \'\').replace(\'.snp_failed_freq_ck.list\',\'\')\n            \n            # 存储唯一文件名，字典中的值是文件名列表\n            if new_filename